In [1]:
import cupy as cp
from cupyx import scatter_add
import cupyx.scipy.sparse.linalg
import cupyx.scipy.sparse as cusparse
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from includes.preprocessor import write_keywords,write_birth,write_parameters
from includes.gamma import domain_mgr, heat_solve_mgr,load_toolpath,get_toolpath
%matplotlib notebook
cp.cuda.Device(1).use()
!nvidia-smi
import pyvista as pv
from pyvirtualdisplay import Display
import vtk

Sun May 23 20:20:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.56       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:01:00.0 Off |                  Off |
| 34%   41C    P8    28W / 260W |    195MiB / 48598MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     On   | 00000000:21:00.0 Off |                  Off |
| 33%   

In [2]:
domain = domain_mgr(filename='input_files/thinwall.k',toolpath_file='input_files/thinwall_toolpath.crs')

Time of reading input files: 0.5253756046295166
Time of calculating critical timestep: 0.32212162017822266
Time of reading and interpolating toolpath: 0.020186901092529297
Number of nodes: 40444
Number of elements: 34192
Number of time-steps: 25065
Time of generating surface: 27.462641954421997


In [3]:
def elastic_stiff_matrix(elements, nodes, shear_0, bulk_0):
    n_n = nodes.shape[0]
    n_e = elements.shape[0]
    n_p = elements.shape[1]
    n_q = 8
    n_int = n_e*n_q
    nodes_pos = nodes[elements]
    Jac = cp.matmul(domain.Bip_ele,nodes_pos[:,cp.newaxis,:,:].repeat(8,axis=1)) # J = B*x [B:8(nGP)*3(dim)*8(nN), x:nE*8*8*3]
    ele_detJac = cp.linalg.det(Jac)
    iJac = cp.linalg.inv(Jac) #inv J (nE*nGp*dim*dim)
    ele_gradN = cp.matmul(iJac,domain.Bip_ele) # dN/dx = inv(J)*B

    ele_B = cp.zeros([n_e,n_q,6,n_p*3])
    ele_B[:,:,0,0:24:3] = ele_gradN[:,:,0,:]
    ele_B[:,:,1,1:24:3] = ele_gradN[:,:,1,:]
    ele_B[:,:,2,2:24:3] = ele_gradN[:,:,2,:]
    ele_B[:,:,3,0:24:3] = ele_gradN[:,:,1,:]
    ele_B[:,:,3,1:24:3] = ele_gradN[:,:,0,:]
    ele_B[:,:,4,1:24:3] = ele_gradN[:,:,2,:]
    ele_B[:,:,4,2:24:3] = ele_gradN[:,:,1,:]
    ele_B[:,:,5,2:24:3] = ele_gradN[:,:,0,:]
    ele_B[:,:,5,0:24:3] = ele_gradN[:,:,2,:]

    temp = cp.array([[0,1,2]]).repeat(n_p,axis=0).flatten()
    jB = 3*cp.tile(elements[:,cp.newaxis,cp.newaxis,:],(1,n_q,6,1)).repeat(3,axis=3) + temp
    vB = ele_B.reshape(-1,n_p*3)
    jB = jB.reshape(-1,n_p*3)
    iB = cp.arange(0,jB.shape[0])[:,cp.newaxis].repeat(n_p*3,axis=1)
    B = cusparse.csr_matrix((cp.ndarray.flatten(vB),(cp.ndarray.flatten(iB), cp.ndarray.flatten(jB))), shape = (6*n_int, 3*n_n), dtype = cp.float)

    IOTA = cp.array([[1],[1],[1],[0],[0],[0]]) 
    VOL = cp.matmul(IOTA,IOTA.transpose()) 
    DEV = cp.diag([1,1,1,1/2,1/2,1/2])-VOL/3

    ELASTC = 2*DEV*shear_0 + VOL*bulk_0
    ele_D = ele_detJac[:,:,cp.newaxis,cp.newaxis]*ELASTC
    temp = cp.arange(0,n_e*n_q*6).reshape(n_e,n_q,6)
    iD = temp[:,:,cp.newaxis,:].repeat(6,axis = 2)
    jD = temp[:,:,:,cp.newaxis].repeat(6,axis = 3)

    D = cusparse.csr_matrix((cp.ndarray.flatten(ele_D),(cp.ndarray.flatten(iD), cp.ndarray.flatten(jD))), shape = (6*n_int, 6*n_int), dtype = cp.float)
    ele_K =  ele_B.transpose([0,1,3,2])@ele_D@ele_B
    ele_K = ele_K.sum(axis = 1)

#     temp = cp.array([[0,1,2]]).repeat(n_p,axis=0).flatten()[:,cp.newaxis]
#     iK = 3*elements[:,:,cp.newaxis].repeat(3,axis=1) + temp
#     iK = iK.repeat(3*n_p,axis=2)
#     jK = 3*elements[:,cp.newaxis,:].repeat(3,axis=2) + temp.transpose()
#     jK = jK.repeat(3*n_p,axis=1)
#     K  = cusparse.csr_matrix((cp.ndarray.flatten(ele_K),(cp.ndarray.flatten(iK),cp.ndarray.flatten(jK))), shape = (3*n_n, 3*n_n), dtype = cp.float)
    K = B.transpose()*D*B 
    return K,B,D,ele_B,ele_D,iD,jD,ele_detJac

def constitutive_problem(E, Ep_prev, Hard_prev, shear_0, bulk_0, a_0, Y):
    IOTA = cp.array([[1],[1],[1],[0],[0],[0]])  
    VOL = cp.matmul(IOTA,IOTA.transpose()) 
    DEV = cp.diag([1,1,1,1/2,1/2,1/2])-VOL/3
    E_tr = E-Ep_prev  
    ELASTC = 2*DEV*shear_0 + VOL*bulk_0
    S_tr = (ELASTC @ E_tr[:,:,:,cp.newaxis]).squeeze()
    SD_tr = (2*DEV*shear_0@E_tr[:,:,:,cp.newaxis]).squeeze() - Hard_prev
    norm_SD = cp.sqrt(cp.sum(SD_tr[:,:,0:3]*SD_tr[:,:,0:3], axis=2)+2*cp.sum(SD_tr[:,:,3:6]*SD_tr[:,:,3:6], axis=2))

    CRIT = norm_SD-Y
    IND_p = CRIT>0 

    S = cp.array(S_tr)
    DS = cp.ones((S.shape[0],S.shape[1],6,6))*ELASTC

    if not IND_p[IND_p].shape[0]:
        Ep = cp.array(Ep_prev)
        Hard = cp.array(Hard_prev)
        return S, DS, IND_p, Ep, Hard   
        
    N_hat = SD_tr[IND_p]/norm_SD[IND_p][:,cp.newaxis].repeat(6,axis=1)  
    denom =  2*shear_0 + a_0 
    Lambda = CRIT[IND_p]/denom

    S[IND_p] = S[IND_p] - 2*N_hat*shear_0*Lambda[:,cp.newaxis].repeat(6,axis=1)  
    NN_hat = N_hat[:,:,cp.newaxis]@N_hat[:,cp.newaxis,:]
    const = 4*shear_0**2/denom

    DS[IND_p] = DS[IND_p] - const*DEV + (const*Y[IND_p]/norm_SD[IND_p])[:,cp.newaxis,cp.newaxis].repeat(6,axis=1).repeat(6,axis=2)*(DEV-NN_hat)


    Ep = cp.array(Ep_prev)
    Ep[IND_p] = Ep[IND_p]+cp.matmul(cp.array([[1],[1],[1],[2],[2],[2]]),Lambda[cp.newaxis]).transpose()*N_hat

    Hard = cp.array(Hard_prev)
    Hard[IND_p] = Hard[IND_p]+(a_0*Lambda[:,cp.newaxis].repeat(6,axis=1))*N_hat
    
    return S, DS, IND_p, Ep, Hard

In [12]:
nodes = domain.nodes
elements = domain.elements
n_n = len(domain.nodes)
n_e = len(domain.elements)
n_p = 8
n_q = 8
n_int = n_e * n_q 
HatP = domain.Nip_ele.transpose()     # HatP   - values of basis functions at the quadrature points,
                                                          # size(HatP)=(n_p,n_q)
DHatP1 = domain.Bip_ele[:,0,:].transpose()        # DHatP1 - derivatives of basis functions at the quadrature points 
                                                          #  in the direction xi_1, size(DHatP1)=(n_p,n_q)
DHatP2 = domain.Bip_ele[:,1,:].transpose()       # DHatP2 - derivatives of basis functions at the quadrature points 
                                                          #  in the direction xi_2, size(DHatP2)=(n_p,n_q)
DHatP3 = domain.Bip_ele[:,2,:].transpose()         # DHatP3 - derivatives of basis functions at the quadrature points 
                                                          #  in the direction xi_3, size(DHatP3)=(n_p,n_q)
                                                          # n_p    - number of basis functions
                                                          # n_q    - number of integration points within one elemen

# values of elastic material parameters
#young = 206900                        # Young's modulus in MPa
young = 116888                         # Young's modulus at average temperature
poisson =  0.32                        # Poisson's ratio
shear_0 = young/(2*(1+poisson))        # shear modulus
bulk_0 = young/(3*(1-2*poisson))       # bulk modulus
# plastic material parematers
#a_0 = 10000
a_0 = 9500                             # Kinematic hardening parameter
#Y_0 = 450*np.sqrt(2/3)                # Yield stress in MPa
Y_0 = 744                              # Note that this is actually yield stress * sqrt(2/3)
T_Ref = 400                            # Reference or Ambient Temparature, always equal to 300 if specified otherwise
#T_Ref = 252                           # Reference or Ambient Temparature, always equal to 300 if specified otherwise
# Tolerence for Newton stopping criterion
tol = 1.0e-8                           # non-dimensionalized tolerence 
# Maximum Number of N_R Iterations allowed
Maxit = 150

# Initialization for the whole boundary-value problem
E = cp.zeros((n_e,n_q,6))                        # strain tensors at integration points
Ep_prev = cp.zeros((n_e,n_q,6))                  # plastic strain tensors at integration points
Hard_prev = cp.zeros((n_e,n_q,6))
Y = Y_0 * cp.ones((n_e,n_q))
U = cp.zeros((n_n,3))
F = cp.zeros((n_n,3))
f = cp.zeros((n_n,3))                          
scl = 1.14875854e-05                           # average thermal expansion coefficient for Ti64A              
alpha_Th = cp.array([scl,scl,scl,0,0,0])

In [13]:
%%time
heat_solver = heat_solve_mgr(domain)
domain.current_time = 0
endtime = 5
timestep = int(endtime/domain.dt)+1
heat_solver.q_in = 200
file_num = 0

layer_time = [11,23,35,47,59,71,83,95,107,119]
layer = 0
active_ele_mech = domain.element_birth < layer_time[layer]
active_elements = elements[active_ele_mech]
active_nodes_mech = domain.node_birth < layer_time[layer]
Q = cp.zeros(nodes.shape,dtype=bool)
Q_T = cp.zeros((n_e,n_q,6),dtype=bool)
Q[active_nodes_mech,:] = 1
Q_N = cp.array(Q)
idirich = cp.array(nodes[:, 2] == -20.0)     
Q[idirich,:] = 0
Q_T[active_ele_mech,:] = 1
K_elast,B,D_elast,ele_B,ele_D,iD,jD,ele_detJac = elastic_stiff_matrix(active_elements, nodes, shear_0, bulk_0)

for t in range(1,timestep):
    if t % 5000 == 0:
        mempool = cp.get_default_memory_pool()
        mempool.free_all_blocks()
        print("Current time:  {}, Percentage done:  {}%".format(domain.current_time,100*t/timestep))  
    heat_solver.time_integration()
    
    if t % 20 == 0:
        if domain.current_time>layer_time[0]:
            layer = layer + 1
            active_ele_mech = domain.element_birth < layer_time[layer]
            active_nodes_mech = domain.node_birth < layer_time[layer]
            Q = cp.zeros(nodes.shape,dtype=bool)
            Q_T = cp.zeros((n_e,n_q,6),dtype=bool)
            Q[active_nodes_mech,:] = 1
            Q_N = cp.array(Q)
            idirich = cp.array(nodes[:, 2] == -20.0)     
            Q[idirich,:] = 0
            Q_T[active_ele_mech,:] = 1
            K_elast,B,D_elast,ele_B,ele_D,iD,jD,ele_detJac = elastic_stiff_matrix(active_elements, nodes, shear_0, bulk_0)
            
        temperature_nodes = heat_solver.temperature[active_elements] - 300
        temperature_ip = (domain.Nip_ele[:,cp.newaxis,:]@temperature_nodes[:,cp.newaxis,:,cp.newaxis].repeat(8,axis=1))[:,:,0,0]
        it = 0
        while True:
            E[active_ele_mech] = (ele_B@(U[active_elements].reshape(-1,24))[:,cp.newaxis,:,cp.newaxis].repeat(n_q,axis=1)).squeeze()
            E[active_ele_mech] = E[active_ele_mech] - temperature_ip[:,:,cp.newaxis].repeat(6,axis=2)*alpha_Th

            S, DS, IND_p,_,_ = constitutive_problem(E[active_ele_mech], Ep_prev[active_ele_mech], 
                                     Hard_prev[active_ele_mech], shear_0, bulk_0, a_0, Y[active_ele_mech])
            vD = ele_detJac[:,:,cp.newaxis,cp.newaxis].repeat(6,axis=2).repeat(6,axis=3) * DS
            D_p = cusparse.csr_matrix((cp.ndarray.flatten(vD), (cp.ndarray.flatten(iD),cp.ndarray.flatten(jD))), shape = D_elast.shape, dtype = cp.float)
            K_tangent = K_elast + B.transpose()*(D_p-D_elast)*B
            n_plast = len(IND_p[IND_p])
            print(' plastic integration points: ', n_plast, ' of ', IND_p.shape[0]*IND_p.shape[1])
            F = B.transpose() @ ((ele_detJac[:,:,cp.newaxis].repeat(6,axis=2)*S).reshape(-1))
            dU,error = cusparse.linalg.cg(K_tangent,-F,tol=tol)
            U_new = U + dU.reshape(-1,3) 
            q1 = dU@K_elast@dU
            q2 = U.flatten()@K_elast@U.flatten()
            q3 = U_new.flatten()@K_elast@U_new.flatten()
            if q2 == 0 and q3 == 0:
                criterion = 0
            else:
                criterion = q1/(q2+q3)
                print('  stopping criterion=  ', criterion)

            U = cp.array(U_new) 
            # test on the stopping criterion
            if  criterion < tol:
                break
                
            # test on number of iteration
            it = it+1
            if  it > Maxit:
                raise Exception('The Newton solver does not converge for the current timestep: {}'.format(t))
                
        E[active_ele_mech] = (ele_B@(U[active_elements].reshape(-1,24))[:,cp.newaxis,:,cp.newaxis].repeat(n_q,axis=1)).squeeze()
        E[active_ele_mech] = E[active_ele_mech] - temperature_ip[:,:,cp.newaxis].repeat(6,axis=2)*alpha_Th
        S, DS, IND_p,Ep,Hard = constitutive_problem(E[active_ele_mech], Ep_prev[active_ele_mech], 
                             Hard_prev[active_ele_mech], shear_0, bulk_0, a_0, Y[active_ele_mech])
        Ep_prev[active_ele_mech] = Ep
        Hard_prev[active_ele_mech] = Hard
        
        if t%200 == 0:
            filename = 'residual_stress/thinwall/thinwall_{}.vtk'.format(file_num)
            save_vtk(filename)
            file_num = file_num + 1

 plastic integration points:  0  of  187136
 plastic integration points:  0  of  187136
 plastic integration points:  0  of  187136
 plastic integration points:  0  of  187136
 plastic integration points:  0  of  187136
 plastic integration points:  0  of  187136
  stopping criterion=   1.0
 plastic integration points:  66  of  187136
  stopping criterion=   0.00017914075693526254
 plastic integration points:  72  of  187136
  stopping criterion=   4.3753786566046745e-07
 plastic integration points:  70  of  187136
  stopping criterion=   4.2509044403463945e-12
 plastic integration points:  11  of  187136
  stopping criterion=   0.18079696018821073
 plastic integration points:  120  of  187136
  stopping criterion=   0.00012928456980023007
 plastic integration points:  126  of  187136
  stopping criterion=   2.2126439298795346e-07
 plastic integration points:  126  of  187136
  stopping criterion=   1.6185872244579448e-13
 plastic integration points:  32  of  187136
  stopping criterio

  stopping criterion=   1.921744174817103e-11
 plastic integration points:  91  of  187136
  stopping criterion=   0.05085586087657513
 plastic integration points:  333  of  187136
  stopping criterion=   0.00015437667384105697
 plastic integration points:  360  of  187136
  stopping criterion=   1.5055818946868666e-06
 plastic integration points:  360  of  187136
  stopping criterion=   2.9302444310007564e-11
 plastic integration points:  107  of  187136
  stopping criterion=   0.04288804637227244
 plastic integration points:  362  of  187136
  stopping criterion=   0.0001381763681765728
 plastic integration points:  376  of  187136
  stopping criterion=   4.1776431280246713e-07
 plastic integration points:  377  of  187136
  stopping criterion=   5.3905819546852646e-11
 plastic integration points:  116  of  187136
  stopping criterion=   0.04318799396948609
 plastic integration points:  355  of  187136
  stopping criterion=   0.0001252634893905715
 plastic integration points:  374  o

 plastic integration points:  738  of  187136
  stopping criterion=   0.00014911831840447038
 plastic integration points:  754  of  187136
  stopping criterion=   5.87880705393569e-07
 plastic integration points:  754  of  187136
  stopping criterion=   6.726166455580412e-12
 plastic integration points:  235  of  187136
  stopping criterion=   0.03877716244172501
 plastic integration points:  772  of  187136
  stopping criterion=   9.532684617772735e-05
 plastic integration points:  792  of  187136
  stopping criterion=   4.6135174461442855e-07
 plastic integration points:  794  of  187136
  stopping criterion=   9.699862687310422e-12
 plastic integration points:  258  of  187136
  stopping criterion=   0.035861941428570444
 plastic integration points:  810  of  187136
  stopping criterion=   0.00012095918511501459
 plastic integration points:  822  of  187136
  stopping criterion=   4.788565658972371e-07
 plastic integration points:  823  of  187136
  stopping criterion=   4.168649518

In [10]:
def save_vtk(filename):
    active_elements = domain.elements[domain.active_elements].tolist()
    active_cells = np.array([item for sublist in active_elements for item in [8] + sublist])
    active_cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(active_elements))
    points = domain.nodes.get() + U.get()
    Sv =  transformation(cp.sqrt(1/2*((S[:,:,0]-S[:,:,1])**2 + (S[:,:,1]-S[:,:,2])**2 + (S[:,:,2]-S[:,:,0])**2 + 6*(S[:,:,3]**2+S[:,:,4]**2+S[:,:,5]**2))), elements[active_ele_mech], ele_detJac, active_nodes_mech)
    S11 = transformation(S[:,:,0], elements[active_ele_mech], ele_detJac, active_nodes_mech)
    S22 = transformation(S[:,:,1], elements[active_ele_mech], ele_detJac, active_nodes_mech)
    S33 = transformation(S[:,:,2], elements[active_ele_mech], ele_detJac, active_nodes_mech)
    S12 = transformation(S[:,:,3], elements[active_ele_mech], ele_detJac, active_nodes_mech)
    S23 = transformation(S[:,:,4], elements[active_ele_mech], ele_detJac, active_nodes_mech)
    S13 = transformation(S[:,:,5], elements[active_ele_mech], ele_detJac, active_nodes_mech)
    active_grid = pv.UnstructuredGrid(active_cells, active_cell_type, points)
    active_grid.point_arrays['temp'] = heat_solver.temperature.get()
    active_grid.point_arrays['S11'] = S11.get()
    active_grid.point_arrays['S22'] = S22.get()
    active_grid.point_arrays['S33'] = S33.get()
    active_grid.point_arrays['S12'] = S12.get()
    active_grid.point_arrays['S23'] = S23.get()
    active_grid.point_arrays['S13'] = S13.get()
    active_grid.save(filename)

In [6]:
def transformation(Q_int, active_elements, ele_detJac, active_nodes_mech):
    Q_int = Q_int.reshape(1,-1)
    elem = cp.array(active_elements.transpose())                      # elements.transpose() with shape (n_p=8,n_e)
    weight = ele_detJac.reshape(1,-1)
    #n_n = COORD.shape[1]          # number of nodes including midpoints
    n_e = elem.shape[1]            # number of elements
    n_p = 8                        # number of vertices per element
    n_q = 8                        # number of quadrature points
    n_int = n_e*n_q                # total number of integrations points
    # values at integration points, shape(vF1)=shape(vF2)=(n_p,n_int)   
    vF1 = cp.matmul(cp.ones((n_p,1)), weight*Q_int)    
    vF2 = cp.matmul(cp.ones((n_p,1)),weight)

    # row and column indices, shape(iF)=shape(jF)=(n_p,n_int)   
    iF = cp.zeros((n_p,n_int), dtype=cp.int32)         ######
    jF = cp.kron(elem, cp.ones((1,n_q), dtype=cp.int32))

    # the asssembling by using the sparse command - values v for duplicate
    # doubles i,j are automatically added together
    F1 = cusparse.csr_matrix((cp.ndarray.flatten(vF1.transpose()),(cp.ndarray.flatten(iF.transpose()), cp.ndarray.flatten(jF.transpose()))), dtype = cp.float) 
    F2 = cusparse.csr_matrix((cp.ndarray.flatten(vF2.transpose()),(cp.ndarray.flatten(iF.transpose()), cp.ndarray.flatten(jF.transpose()))), dtype = cp.float) 

    #
    # Approximated values of the function Q at nodes of the FE mesh
    #
    Q = cp.array(F1/F2)
    Q_node = cp.ones(Q.shape[1])
    Q_node[active_nodes_mech] = Q[0,active_nodes_mech]
    return Q_node

In [ ]:
domain.dt*200

In [ ]:
points.shape

In [ ]:
U.shape